#Import

In [0]:
!rm -rf mola
!git clone https://github.com/dbt-ethz/mola.git

Cloning into 'mola'...
remote: Enumerating objects: 124, done.
remote: Counting objects: 100% (124/124), done.
remote: Compressing objects: 100% (119/119), done.
remote: Total 1405 (delta 65), reused 13 (delta 4), pack-reused 1281
Receiving objects: 100% (1405/1405), 24.69 MiB | 14.41 MiB/s, done.
Resolving deltas: 100% (891/891), done.


In [0]:
from IPython.display import HTML
import random
from mola.core import *
from mola.grid import Grid
import mola.factory as factory
import mola.io as io


import mola.renderBabylonJS as r3D
import mola.subdivision as subdivision

##Polyomino Definition

In [0]:
class Polyomino:
    def __init__(self,vertices,color=(1,1,1,1)):
      self.vertices=vertices
      self.color=color
      self.normalise()

    def normalise(self):
      self.bounds=Box()
      for v in self.vertices:
          self.bounds.addPoint(v.x,v.y,v.z)
      for v in self.vertices:
          v.x=v.x-self.bounds.x1
          v.y=v.y-self.bounds.y1
          v.z=v.z-self.bounds.z1
      self.bounds=Box()
      for v in self.vertices:
          self.bounds.addPoint(v.x,v.y,v.z)
    
    def getMesh(self,dim=10):
      polyMesh=Mesh()
      for v in self.vertices:
        boxMesh=factory.constructBox(v.x*dim,v.y*dim,v.z*dim,(v.x+1)*dim,(v.y+1)*dim,(v.z+1)*dim)
        polyMesh.faces.extend(boxMesh.faces)
      for f in polyMesh.faces:
        f.color=self.color
      return polyMesh

##Generate & Test

In [0]:
def findBestPosition(polyomino):
    #polymino searches for the lowest position in the grid where it could be placed without conflict
    global grid
    for z in range(grid.nZ):
        for y in range(grid.nY):
            for x in range(grid.nX):
                fits=True
                for v in polyomino.vertices:
                    cX=x+v.x
                    cY=y+v.y
                    cZ=z+v.z
                    if cX<grid.nX and cY<grid.nY and cZ<grid.nZ:
                        if grid.get(cX,cY,cZ)!=0:
                            fits=False
                            break
                    else: 
                        fits=False
                        break
                if fits:
                    for v in polyomino.vertices:
                        cX=x+v.x
                        cY=y+v.y
                        cZ=z+v.z
                        grid.set(polyomino,cX,cY,cZ)
                        polyomino.position=Vertex(cX,cY,cZ)
                    return True
    return False

def test():
    #fill all polyominoes inside
    grid.values=[0]*grid.nX*grid.nY*grid.nZ
    for p in polyominos:
        findBestPosition(p)
    centerOfMass=Vertex()
    # calculate fitness by density
    nNodes=0
    for z in range(grid.nZ):
        for y in range(grid.nY):
            for x in range(grid.nX):
                if grid.get(x,y,z)!=0:
                    nNodes+=1
    if nNodes==0:return 100000
    return grid.nX*grid.nY*grid.nZ-nNodes

def backup():
    #save the sequence of polyminos
    global backUpList,polyominos
    backUpList=list(polyominos)

def restore():
    # restore the old sequence
    global backUpList,polyominos
    polyominos=list(backUpList)
    test()

def mutate(mutationRate=1):
    # mutate the sequence
    global polyominos
    for i in range(mutationRate):
        rnd1=random.randint(0,len(polyominos)-1)
        rnd2=random.randint(0,len(polyominos)-1)
        temp=polyominos[rnd1]
        polyominos[rnd1]=polyominos[rnd2]
        polyominos[rnd2]=temp

def mutateForBetter(mutationRate=1):
    global fitness
    backup()
    mutate(mutationRate)
    newFitness=test()
    if newFitness>fitness:
        restore()
    else:
        fitness=newFitness

#Exercise 1.1

Define your own environment (stay below 20.000 Voxels). Create a mix of 4 different types of polyominoes and export them as OBJ

Naming: lastname_type_X.obj 

## Define Types

In [0]:
def constructRandomType():
  vertices=[]
  for j in range(8):
    x=random.randint(0,3)
    y=random.randint(0,3)
    z=random.randint(0,3)
    v=Vertex(x,y,z)
    if v not in vertices:
       vertices.append(v)
  polymino=Polyomino(vertices)
  polymino.color=(random.uniform(0,1),random.uniform(0,1),random.uniform(0,1),1)
  return polymino

def constructRowType():
  vertices=[]
  for j in range(8):
    v=Vertex(j,0,0)
    vertices.append(v)
  polymino=Polyomino(vertices)
  polymino.color=(random.uniform(0,1),random.uniform(0,1),random.uniform(0,1),1)
  return polymino

def constructRingType():
  """x z coordinates"""
  ####
  #  #
  ####
  vertices=[]
  for j in range(4):
    v=Vertex(j,0,0)
    vertices.append(v)
    v=Vertex(j,0,2)
    vertices.append(v)
  vertices.append(Vertex(0,0,1))
  vertices.append(Vertex(3,0,1))
  polymino=Polyomino(vertices)
  polymino.color=(1,1,0,1)
  return polymino

def constructExtrudedRingType():
  """x z coordinates extruded in y direction"""
  ####
  #  #
  ####
  vertices=[]
  for y in range(3):
    for j in range(4):
      v=Vertex(j,y,0)
      vertices.append(v)
      v=Vertex(j,y,2)
      vertices.append(v)
    vertices.append(Vertex(0,y,1))
    vertices.append(Vertex(3,y,1))
  polymino=Polyomino(vertices)
  polymino.color=(0,1,1,1)
  return polymino

def constructType1():
  vertices=[]
  vertices.append(Vertex(0,0,0))
  vertices.append(Vertex(1,0,0))
  vertices.append(Vertex(1,1,0))
  vertices.append(Vertex(2,0,0))
  vertices.append(Vertex(2,0,1))
  polymino=Polyomino(vertices)
  polymino.color=(1,0,0,1)
  return polymino

def constructType2():
  vertices=[]
  vertices.append(Vertex(0,0,0))
  vertices.append(Vertex(1,0,0))
  vertices.append(Vertex(1,1,0))
  vertices.append(Vertex(2,0,0))
  vertices.append(Vertex(2,0,1))
  vertices.append(Vertex(3,0,1))
  polymino=Polyomino(vertices)
  polymino.color=(0,1,0,1)
  return polymino

def constructType3():
  vertices=[]
  vertices.append(Vertex(0,1,0))
  vertices.append(Vertex(1,0,0))
  vertices.append(Vertex(1,1,0))
  vertices.append(Vertex(2,0,0))
  vertices.append(Vertex(2,0,1))
  vertices.append(Vertex(3,0,1))
  polymino=Polyomino(vertices)
  polymino.color=(0,0,1,1)
  return polymino

##Viewer

In [0]:
type1=constructExtrudedRingType()
#export the type as OBJ
io.exportOBJ(type1.getMesh(),'type1.obj')
#display the type
HTML(r3D.displayMesh(polyMesh,True,True))

NameError: ignored

## Setup 

In [0]:
global grid,fitness,polyominos

# define grid
grid=Grid(16,4,48)
grid.values=[0]*grid.nX*grid.nY*grid.nZ

#define polyominos
polyominos=[]
  
# here we create 20 times the same polyomino from the initial type
polyType=constructRandomType()
unitsPerType=20
for n in range(unitsPerType):
    polymino=Polyomino(polyType.vertices)
    polymino.color=polyType.color
    polyominos.append(polymino)

# this version simply creates x times type3 and stores it in the list
for n in range(30):
  polymino=constructExtrudedRingType()
  polyominos.append(polymino)

for n in range(30):
  polymino=constructRingType()
  polyominos.append(polymino)

for n in range(20):
  polymino=constructType1()
  polyominos.append(polymino)

# Exercise 1.2

Export initial configuration and an optimized version as OBJ

Naming: lastname_init.obj and lastname_optimized.obj

##Optimize

In [0]:
#random.shuffle(polyominos)
fitness=test()
generation=0
print('generation: '+str(generation)+'  fitness: '+str(fitness))

In [0]:
for i in range(20):
  generation+=1
  mutateForBetter(3)
print('generation: '+str(generation)+'  fitness: '+str(fitness))

##Display & Export

In [0]:
# get mesh from grid
mesh=Mesh()
faces=[]
for p in polyominos:
  functionIn= lambda value: 1 if value==p else 0
  functionOut= lambda value: 1 if value!=p else 0
  cmesh=grid.getQuadMesh(functionIn,functionOut)
  for face in cmesh.faces:
    face.color=p.color
  mesh.faces.extend(cmesh.faces)
box=mesh.getBounds()
mesh.weldVertices()
mesh.translate(-box.getCenterX(),-box.getCenterY(),-box.getCenterZ())

#io.exportOBJ(mesh,'test.obj')
HTML(r3D.displayMesh(mesh,False,True))

# Exercise 2 (Plus)

Option1: Apply a shape grammar to the meshes before exporting

Option2: Introduce fixed obstacles

In [0]:
# get mesh from grid
mesh=Mesh()
faces=[]
for p in polyominos:
  # lambda is a way to create an anonymous function
  functionIn= lambda value: 1 if value==p else 0
  functionOut= lambda value: 1 if value!=p else 0
  cmesh=grid.getQuadMesh(functionIn,functionOut)
  for face in cmesh.faces:
    face.color=p.color
  # here you can apply your shape grammar
  mesh.faces.extend(cmesh.faces)
mesh.weldVertices()
box=mesh.getBounds()
mesh.translate(-box.getCenterX(),-box.getCenterY(),-box.getCenterZ())

io.exportOBJ(mesh,'test.obj')
HTML(r3D.displayMesh(mesh))